<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/Integrations/Langchain/Chains/Router_chain_examples_with_Clarifai_SDK.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>

# **Router chain with Clarifai SDK prompt templates**

This notebook will walk you through a demo on when and how to use Router chain from langchain in an effective way and leverage it to use for variety of tasks in your AI Apps.

For the purpose of demo we going to create various prompt templates based on the Clarifai-python SDK use cases and test the router chain implementation based on these templates for variety of scenarios.




## **Chains**
While using a language model (LLM) on its own is good for simple tasks, tackling more complex applications often requires connecting multiple LLMs either with each other or with different parts of the system.

LangChain provides various chains frameworks for connecting these components. 
Explore about various [Chains](https://python.langchain.com/docs/modules/chains/) in langchain and also it provides different ways to encompass [memory](https://python.langchain.com/docs/modules/memory/) into the chain which will add additional capabilities to your AI apps.

### Setup
#### Install the latest langchain package and Clarifai package.

In [ ]:
!pip install langchain
!pip install clarifai

You can use several language models from [Clarifai](https://clarifai.com/explore/models?filterData=%5B%7B%22field%22%3A%22use_cases%22%2C%22value%22%3A%5B%22llm%22%5D%7D%5D&page=1&perPage=24) platform. Sign up and get your [PAT](https://clarifai.com/settings/security) to access it.

For our example we are using codellama-34b-instruct model.

##[Codellama](https://huggingface.co/codellama)

* Code Llama is a collection of pretrained and fine-tuned generative text models ranging in scale from 7 billion to 34 billion parameters.
* This model is designed for general code synthesis and understanding.
* This use case where we are trying to modify our code and run it might be one of the specialized use cases for using codellama-34b-instruct model.


In [68]:
MODEL_URL="https://clarifai.com/meta/Llama-2/models/codellama-34b-instruct"

Initialize your PAT key as environment variable.

In [69]:
import os
os.environ["CLARIFAI_PAT"]="YOUR_CLARIFAI_PAT"

set_debug(True)

In [ ]:
from langchain.globals import set_debug
set_debug(True)

Setting the global debug flag will cause all LangChain components with callback support (chains, models, agents, tools, retrievers) to print the inputs they receive and outputs they generate.

### **Clarifai LLM**
You can access different models from clarifai platform within langchain framework. By signing up to clarifai and getting a clarifai's PAT token you can access LLM models, embedding models.

In [70]:
from langchain.llms import Clarifai
llm=Clarifai(model_url=MODEL_URL)

In [71]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

### **Prompt Templates**

For the demo of router chains we are creating different types of prompt templates which gives us an executable code of our own [Clarifai-Python](https://github.com/Clarifai/clarifai-python) SDK. These are used to perform simple mundane tasks with clarifai platform like creating/deleting apps, datasets, ingestion of imputs and prediction of concepts.

In [72]:
ingest_input_image_template= PromptTemplate(input_variables=["input"], template=("""
You are an programmer with deep understanding of python code, If user wants to ingest image data/inputs into their clarifai app,\
then you need to carefully go through the user query and retrieve certain parameters needed for the below code to execute.\
retrieve userid, appid and image_url from the user Input : {input} and fill the parameters and return below code output which is ready to be executed.\
Check further if the user wants to ingest list of inputs, if so use for loop inside code to upload.
```python
from clarifai.client.user import User
clarifai_app = User(User_id).app(App_id)
input_obj = app.inputs()
res=input_obj.upload_from_url(input_id="id-1", image_url=url)
print("Inputs added successfully")
````
Don't give any alternate suggestions, return only the code in above format with minor changes.!
"""),
)

ingest_input_text_template = PromptTemplate.from_template( """
You are an programmer with deep understanding of python code, If user wants to ingest text data/inputs into their clarifai app,\
then you need to carefully go through the user query and retrieve certain parameters needed for the below code to execute.\
retrieve userid, appid and text_url from the user Input : {input} and fill the parameters and return below code output which is ready to be executed.\
Check further if the user wants to ingest list of inputs, if so use for loop inside code to upload.
```python
from clarifai.client.user import User
clarifai_app = User(User_id).app(App_id)
input_obj = app.inputs()
res=input_obj.upload_from_url(input_id="i", text_url=url)
print("Inputs added successfully")
````
Don't give any alternate suggestions, return only the code in above format with minor changes.!
"""
)

create_app_template= PromptTemplate.from_template("""
As a Python programmer with a deep understanding of code, your task is to help users to create their Clarifai app using the Clarifai Python SDK.
Carefully go through the user's query and retrieve the necessary parameters from user's prompt for the below provided code snippet and fill the parameters\
and return below code which is ready to be executed.
For creating apps make sure the user's : {input} consists of 'user_id', 'app_id', 'base_workflow'(optional).
```python
from clarifai.client.user import User
client = User(user_id="user_id")
app = client.create_app(app_id="app_id", base_workflow="Universal")
print("App created successfully")
```
Don't give any alternate suggestions, return only the code in above format with minor changes.!
 """)

delete_app_template=PromptTemplate.from_template("""
As a Python programmer with a deep understanding of code, your task is to help users to delete Clarifai app using the Clarifai Python SDK.
Carefully go through the user's query and retrieve the necessary parameters from user's prompt for the below provided code snippet and fill the parameters\
and return below python code which is ready to be executed.
For deleting dataset make sure the user's : {input} consists of 'user_id', 'app_id'.
```python
from clarifai.client.user import User
user = User("user_id").delete_app("app_id")
print("App deleted successfully")
```
Don't give any alternate suggestions, return only the code in above format with minor changes.!
""")

create_dataset_template= PromptTemplate.from_template("""
As a Python programmer with a deep understanding of code, your task is to help users to create dataset inside Clarifai app using the Clarifai Python SDK.
Carefully go through the user's query and retrieve the necessary parameters from user's prompt for the below provided code snippet and fill the parameters\
and return below code which is ready to be executed.
For creating dataset make sure the user's : {input} consists of 'user_id', 'app_id', 'dataset_id'.
```python
from clarifai.client.app import App
app = App(app_id="app_id", user_id="user_id")
dataset = app.create_dataset(dataset_id="dataset_id")
print("dataset created successfully)
```
Don't give any alternate suggestions, return only the code in above format with minor changes.!
""")

delete_dataset_template= PromptTemplate.from_template(
"""
As a Python programmer with a deep understanding of code, your task is to help users to delete dataset inside Clarifai app using the Clarifai Python SDK.
Carefully go through the user's query and retrieve the necessary parameters from user's prompt for the below provided code snippet and fill the parameters\
and return below code which is ready to be executed.
For deleting dataset make sure the user's : {input} consists of 'user_id', 'app_id', 'dataset_id'.
Don't give any alternate suggestions, return only the code in above format with minor changes.!
```python
from clarifai.client.app import App
app = App(app_id="app_id", user_id="user_id")
app.delete_dataset(dataset_id="dataset_id")
print("dataset deleted successfully)
```
Don't give any alternate suggestions, return only the code in above format with minor changes.!
"""
)

predict_model_template= PromptTemplate.from_template(
"""
As a Python programmer with a deep understanding of code, your task is to help users to predict concepts from image using the Clarifai Python SDK.
Carefully go through the user's query and retrieve the necessary parameters from user's prompt for the below provided code snippet and fill the parameters\
and return below code which is ready to be executed.
For predicting concepts using clarifai SDK make sure the user's : {input} consists of 'model_url'(optional) and 'image_url'.
If no model_url is given use Example model_url: https://clarifai.com/clarifai/main/models/general-image-recognition.
```python
from clarifai.client.model import Model
model = Model("model_url")
model_prediction = model.predict_by_url(url='image_url', input_type='image')
for concept in model_prediction.outputs[0].data.concepts:
  print('name :', concept.name, ',', 'score:', concept.value)
```
Don't give any alternate suggestions, return only the code in above format with minor changes.!
"""
)




### Chains from prompt templates

So now that we have prompt templates for various scenarios, the next step is to create a proper chains based on these templates. Make sure to give appropriate description for each prompts.

In [73]:
#Prompt infos store the description and template details of the corresponding stage. We can add more descriptions to obtain crispy results.

prompt_infos=[
    {
        "name" : "Ingest image inputs",
        "description" : "When Image inputs needs to be ingested into clarifai app",
        "prompt_template" : ingest_input_image_template,
    },
    {
        "name" : "Ingest text inputs",
        "description" : "To ingest text inputs into clarifai app",
        "prompt_template" : ingest_input_text_template,
    },
    {
        "name" : "create clarifai app",
        "description" : "To create clarifai app for the user",
        "prompt_template" : create_app_template,
    },
    {
        "name" : "delete clarifai app",
        "description" : "To delete clarifai app for the user",
        "prompt_template" : delete_app_template,
    },
    {
        "name" : "create dataset",
        "description" : "To create dataset inside clarifai app for the user",
        "prompt_template" : create_dataset_template,
    },
    {
        "name" : "delete dataset",
        "description" : "To delete dataset inside clarifai app for the user",
        "prompt_template" : delete_dataset_template,
    },
    {
        "name" : "predict concepts",
        "description" : "To predict concepts for the given image using clarifai model URL",
        "prompt_template" : predict_model_template,
    }

]

#storing the prompt template name and chain info in a dictionary.
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    chain = LLMChain(llm=llm, prompt=prompt_template)
    destination_chains[name] = chain

# In case if user's query is not based on the above templates we need to make sure to have a base default chain to address such conditions.
default_chain = ConversationChain(llm=llm, output_key="query")

### Creating LLMrouterChain


After we have our chains created for each prompt templates, we'll create a router chain class which helps routing  queries to appropriate prompt templates.

In [74]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE


destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)


router_chain = LLMRouterChain.from_llm(llm, router_prompt)
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

Ingest image inputs: When Image inputs needs to be ingested into clarifai app
Ingest text inputs: To ingest text inputs into clarifai app
create clarifai app: To create clarifai app for the user
delete clarifai app: To delete clarifai app for the user
create dataset: To create dataset inside clarifai app for the user
delete dataset: To delete dataset inside clarifai app for the user
predict concepts: To predict concepts for the given image using clarifai model URL


Custom function to prepare our LLM response for post processing. We will pass the output into the function to achieve our task.

In [75]:
def runquery(llmresponse):
  _, after = llmresponse.split("```python")
  code=after.split("```")[0]
  exec(code)

###Observations
Let's dive into predictions part where we'll test our Router pipeline with several scenarios and check if it is routing to appropriate prompt templates to generate response.

In [76]:
response=chain.run("I want to create clarifai app 'my_app_2' in my account 'mogith-p-n'?")
print(response)
runquery(response)



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


create clarifai app: {'input': 'my_app_2 mogith-p-n'}
> Finished chain.


### Solution

```python
from clarifai.client.user import User
client = User(user_id="mogith-p-n")
app = client.create_app(app_id="my_app_2", base_workflow="Universal")
print("App created successfully")
```



2023-12-19 14:17:08 INFO     clarifai.client.user:                                                      ]8;id=237050;file:///usr/local/lib/python3.10/dist-packages/clarifai/client/user.py\user.py]8;;\:]8;id=652191;file:///usr/local/lib/python3.10/dist-packages/clarifai/client/user.py#122\122]8;;\
                             App created                                                                           
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "8502d132dfe77c09f984a41694bdd03d"                                            
                                                                                                                   

2023-12-19 14:17:08 INFO     clarifai.client.user:                                                      ]8;id=493885;file:///usr/local/lib/python3.10/dist-packages/clarifai/client/user.py\user.py]8;;\:]8;id=569987;file:///usr/local/lib/python3.10/dist-packages/clarifai/client/user.py#122\122]8;;\
                             App created                                                                           
                             code: SUCCESS                                                                         
                             description: "Ok"                                                                     
                             req_id: "8502d132dfe77c09f984a41694bdd03d"                                            
                                                                                                                   

INFO:clarifai.client.user:
App created
code: SUCCESS
description: "Ok"
req_id: "8502d132dfe77c09f984a41694bdd03d"



App created successfully


We got our clarifai App created successfully with just single line of query.
You can observe how our model just simplifies the task by producing a super executable code which in turn we can pass it to our AI pipeline.

In [ ]:
response=chain.run("I want to delete clarifai app 'my_app' in my account 'mogith-p-n'?")
runquery(response)



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


delete clarifai app: {'input': "I want to delete clarifai app 'my_app' in my account 'mogith-p-n'"}
> Finished chain.
App deleted successfully


Note that the chain actually showcases which Prompt template it has been used for this particular scenario.

Now try to ingest some inputs,

In [ ]:
query="I want to ingest these images https://images.pexels.com/photos/139257/pexels-photo-139257.jpeg, https://images.pexels.com/photos/139247/pexels-photo-139247.jpeg into app 'my_app' in my clarifai account mogith-p-n."
resp=chain.run(query)
runquery(resp)



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Ingest image inputs: {'input': "I want to ingest these images https://images.pexels.com/photos/139257/pexels-photo-139257.jpeg, https://images.pexels.com/photos/139247/pexels-photo-139247.jpeg into app 'my_app' in my clarifai account mogith-p-n."}
> Finished chain.
Image 1 added successfully
Image 2 added successfully


Let's try prediction on image with a simple query,


In [ ]:
query="Detect what's in this image https://images.pexels.com/photos/139257/pexels-photo-139257.jpeg"
resp=chain.run(query)
runquery(resp)



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


predict concepts: {'input': "Detect what's in this image https://images.pexels.com/photos/139257/pexels-photo-139257.jpeg"}
> Finished chain.
name : no person , score: 0.9926819205284119
name : summer , score: 0.984977126121521
name : beach , score: 0.9845458269119263
name : nature , score: 0.9790793657302856
name : sand , score: 0.976331353187561
name : water , score: 0.9747101664543152
name : sky , score: 0.9711887836456299
name : travel , score: 0.9610219597816467
name : tropical , score: 0.9548453688621521
name : sun , score: 0.9491840600967407
name : sea , score: 0.946053683757782
name : sunset , score: 0.9431171417236328
name : outdoors , score: 0.916150689125061
name : composure , score: 0.8943774104118347
name : dune , score: 0.881453812122345
name : exotic , score: 0.8704201579093933
name : relaxation , score: 0.8697872757911682
name : romance , score: 0.86239093542099
name : seashore , score: 0.8560242652893066
name : recreation , score: 0.8548668026924133


How cool is to just detect the concepts in the given image with just simple prompt. by the way the model detect actually uses one of clarifai's benchmark [image detection](https://clarifai.com/clarifai/main/models/general-image-recognition) model to predict concepts.

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://docs.clarifai.com/python-sdk/api-reference](https://docs.clarifai.com/python-sdk/api-reference)

---